In [26]:
import pandas as pd
import datetime
import ast
from fuzzywuzzy import process
from openai import OpenAI


In [3]:
df_games= pd.read_csv('all_teams.csv')
df_posts = pd.read_csv('hockey.csv')

In [7]:
# get posts in target date range and language
after = datetime.date(2023, 10, 10)
before = datetime.date(2024, 4, 18)
df_posts["creation_time"] = pd.to_datetime(df_posts["creation_time"], utc=True)
df_posts = df_posts[(df_posts['creation_time'].dt.date >= after) & (df_posts['creation_time'].dt.date <= before)]
df_posts = df_posts.sort_values(by="creation_time")
df_posts = df_posts[df_posts["lang"] == "en"]

In [9]:
# combine caption and hashtags to use to look for teams
df_posts["hashtags"] = df_posts["hashtags"].fillna(" ")
df_posts["text"] = df_posts["text"].fillna(" ")

df_posts["combined"] = df_posts["hashtags"] + " " + df_posts["text"]

In [8]:
df_games = df_games[df_games['situation'] == 'all']
df_games = df_games[df_games['season'] == 2023]
df_games['gameDate'] = pd.to_datetime(df_games['gameDate'].astype(str), format='%Y%m%d')
df_games['gameDate'] = pd.to_datetime(df_games['gameDate'])

In [11]:
def parse_combined(combined_field):
    try:
        # Try parsing as a list
        return ' '.join(ast.literal_eval(combined_field))
    except (ValueError, SyntaxError):
        # If it fails, treat it as a plain string
        return combined_field.replace("[", "").replace("]", "").replace(",", "").replace("'", "").replace('"', '')

In [12]:
team_mapping = {
    "ANA": "Anaheim Ducks",
    "ARI": "Arizona Coyotes",
    "BOS": "Boston Bruins",
    "BUF": "Buffalo Sabres",
    "CAR": "Carolina Hurricanes",
    "CBJ": "Columbus Blue Jackets",
    "CGY": "Calgary Flames",
    "CHI": "Chicago Blackhawks",
    "COL": "Colorado Avalanche",
    "DAL": "Dallas Stars",
    "DET": "Detroit Red Wings",
    "EDM": "Edmonton Oilers",
    "FLA": "Florida Panthers",
    "LAK": "Los Angeles Kings",
    "MIN": "Minnesota Wild",
    "MTL": "Montreal Canadiens",
    "NJD": "New Jersey Devils",
    "NSH": "Nashville Predators",
    "NYI": "New York Islanders",
    "NYR": "New York Rangers",
    "OTT": "Ottawa Senators",
    "PHI": "Philadelphia Flyers",
    "PIT": "Pittsburgh Penguins",
    "SEA": "Seattle Kraken",
    "SJS": "San Jose Sharks",
    "STL": "St. Louis Blues",
    "TBL": "Tampa Bay Lightning",
    "TOR": "Toronto Maple Leafs",
    "VAN": "Vancouver Canucks",
    "VGK": "Vegas Golden Knights",
    "WPG": "Winnipeg Jets",
    "WSH": "Washington Capitals"
}

city_mapping = {
    "ANA": "Anaheim",
    "ARI": "Arizona",
    "BOS": "Boston",
    "BUF": "Buffalo",
    "CAR": "Carolina",
    "CBJ": "Columbus",
    "CGY": "Calgary",
    "CHI": "Chicago",
    "COL": "Colorado",
    "DAL": "Dallas",
    "DET": "Detroit",
    "EDM": "Edmonton",
    "FLA": "Florida",
    "LAK": "Los Angeles",
    "MIN": "Minnesota",
    "MTL": "Montreal",
    "NJD": "New Jersey",
    "NSH": "Nashville",
    "NYI": "New York",
    "NYR": "New York",
    "OTT": "Ottawa",
    "PHI": "Philadelphia",
    "PIT": "Pittsburgh",
    "SEA": "Seattle",
    "SJS": "San Jose",
    "STL": "St. Louis",
    "TBL": "Tampa Bay",
    "TOR": "Toronto",
    "VAN": "Vancouver",
    "VGK": "Vegas",
    "WPG": "Winnipeg",
    "WSH": "Washington"
}

name_mapping = {
    "ANA": "Ducks",
    "ARI": "Coyotes",
    "BOS": "Bruins",
    "BUF": "Sabres",
    "CAR": "Hurricanes",
    "CBJ": "Blue Jackets",
    "CGY": "Flames",
    "CHI": "Blackhawks",
    "COL": "Avalanche",
    "DAL": "Stars",
    "DET": "Red Wings",
    "EDM": "Oilers",
    "FLA": "Panthers",
    "LAK": "Kings",
    "MIN": "Wild",
    "MTL": "Canadiens",
    "NJD": "Devils",
    "NSH": "Predators",
    "NYI": "Islanders",
    "NYR": "Rangers",
    "OTT": "Senators",
    "PHI": "Flyers",
    "PIT": "Penguins",
    "SEA": "Kraken",
    "SJS": "Sharks",
    "STL": "Blues",
    "TBL": "Lightning",
    "TOR": "Maple Leafs",
    "VAN": "Canucks",
    "VGK": "Golden Knights",
    "WPG": "Jets",
    "WSH": "Capitals"
}


In [13]:
df_games['team_full_name'] = df_games['team'].map(team_mapping)
df_games['city'] = df_games['team'].map(city_mapping)
df_games['name'] = df_games['team'].map(name_mapping)

In [20]:
df_posts['lowered'] = df_posts['combined'].str.lower()

In [21]:
def classify_post(post, games):
    # Match games by date
    post_date = post['creation_time'].date()
    # Create date range for the day before, same day, and day after
    date_range = [
        pd.Timestamp(post_date - pd.Timedelta(days=1)),  # Day before
        pd.Timestamp(post_date),                        # Same day
        pd.Timestamp(post_date + pd.Timedelta(days=1))  # Day after
    ]

    # Filter games by date range
    nearby_games = games[games['gameDate'].isin(date_range)]
    
    # Extract keywords from the post
    post_keywords = parse_combined(post['lowered'])
    
    # Match team names
    for _, game in nearby_games.iterrows():
        if game['team'] in post_keywords:
            return game['gameId']
        team_match = process.extractOne(game['city'].lower(), [post_keywords])
        team_match2 = process.extractOne(game['team_full_name'].lower(), [post_keywords])
        team_match3 = process.extractOne(game['name'].lower(), [post_keywords])

        if team_match and team_match[1] > 55:  # Adjust threshold as needed
            return game['gameId']
        if team_match2 and team_match2[1] > 55:
            return game['gameId']
        if team_match3 and team_match3[1] > 55:
            return game['gameId']
    
    return "No Game"  

# Apply the classification function to posts
df_posts['game'] = df_posts.apply(lambda post: classify_post(post, df_games), axis=1)

In [22]:
df_posts = df_posts[df_posts['game'] != 'No Game']

In [24]:
# Get the teams involved in a game
def get_teams(game_id, games):
    game_info = games[games['gameId'] == game_id]
    if game_info.empty:
        print(f"Game ID {game_id} not found")
        return "Unknown", "Unknown"

    team1 = game_info.iloc[0]['playerTeam']
    team2 = game_info.iloc[0]['opposingTeam']
    if game_info.iloc[0]["goalsFor"] > game_info.iloc[0]["goalsAgainst"]:
        return team1, team2
    else:
        return team2, team1

df_posts[['winning_team', 'losing_team']] = df_posts['game'].apply(lambda game_id: get_teams(game_id, df_games)).apply(pd.Series)

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key='YOUR_API_KEY',
)

In [29]:
def label_text(text, team1, team2):
    prompt =f"""
    You are an expert at understanding sports fan sentiment from social media posts. Below is a post, and your task is to determine which of the following teams the poster is a fan of:{team1, team2} . 
    For each post, output the team they are a fan of. Use the abbreviation for the team. For example, MTL for the Montreal Canadiens. RETURN ONLY THE ABBREVIATION FOR THE TEAM AND NOTHING ELSE. 
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"Please label the following text: '{text}'"}
        ]
    )
    label = response.choices[0].message.content
    return label.strip()

In [ ]:
def check_team_in_post(post):
    winning_team = post['winning_team']
    losing_team = post['losing_team']
    combined_text = post['lowered']
    post_keywords = parse_combined(post['lowered'])
    team_match = process.extractOne(city_mapping[winning_team].lower(), [post_keywords])
    team_match2 = process.extractOne(team_mapping[winning_team].lower(), [post_keywords])
    team_match3 = process.extractOne(name_mapping[winning_team].lower(), [post_keywords])
    team_match4 = process.extractOne(city_mapping[losing_team].lower(), [post_keywords])
    team_match5 = process.extractOne(team_mapping[losing_team].lower(), [post_keywords])
    team_match6 = process.extractOne(name_mapping[losing_team].lower(), [post_keywords])
    win = False
    lose = False
    if team_match and team_match[1] > 55 or team_match2 and team_match2[1] > 55 or team_match3 and team_match3[1] > 55:
        win = True
    
    if team_match4 and team_match4[1] > 55 or team_match5 and team_match5[1] > 55 or team_match6 and team_match6[1] > 55:
        lose = True
    
    if win and lose:
        return label_text(combined_text, winning_team, losing_team)
    elif win:
        return winning_team
    elif lose:
        return losing_team

df_posts['team_mentioned'] = df_posts.apply(check_team_in_post, axis=1)

### Here I manually checked each entry that didn't return with the correct format

df_incorrect = df_use[df_use['team_mentioned'].str.len() <= 3]

### Final dataframe is store in df_cleaned

### Get other features

In [99]:
# win or loss
df_cleaned['team_won'] = (df_cleaned['team_mentioned'] == df_cleaned['winning_team']).astype(int)

In [106]:
# close game and blow out
df_games['diff'] = df_games['goalsFor'] - df_games['goalsAgainst']

games_unique = df_games[['gameId', 'diff']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned['close_game'] = (abs(df_cleaned['diff']) <= 1).astype(int)
df_cleaned['blowout'] = (abs(df_cleaned['diff']) >=4).astype(int)
df_cleaned.drop(columns=['diff', 'gameId'], inplace=True)


In [107]:
# high scoring
df_games['totalGoals'] = df_games['goalsFor'] + df_games['goalsAgainst']

games_unique = df_games[['gameId', 'totalGoals']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned['highScoring'] = (abs(df_cleaned['totalGoals']) >= 1).astype(int)
df_cleaned.drop(columns=['totalGoals', 'gameId'], inplace=True)

In [141]:
# above 500
# Initialize the win count column
df_games['win_count'] = 0
df_games['loss_count'] = 0

# Iterate through each team and calculate the cumulative win count
for team in df_games['playerTeam'].unique():
    team_games = df_games[df_games['playerTeam'] == team].sort_values(by='gameDate')
    team_games['win_count'] = team_games['goalsFor'] > team_games['goalsAgainst']
    team_games['win_count'] = team_games['win_count'].cumsum()
    team_games['loss_count'] = team_games['goalsFor'] < team_games['goalsAgainst']
    team_games['loss_count'] = team_games['loss_count'].cumsum()
    df_games.loc[team_games.index, 'loss_count'] = team_games['loss_count']
    df_games.loc[team_games.index, 'win_count'] = team_games['win_count']
df_games['winPercentage'] = df_games['win_count'] / (df_games['win_count'] + df_games['loss_count'])

games_unique = df_games[['gameId', 'winPercentage']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned['above500'] = (df_cleaned['winPercentage'] > 0.5).astype(int)
df_cleaned.drop(columns=['winPercentage', 'gameId'], inplace=True)

In [121]:
# days since last game
df_games = df_games.sort_values(by='gameDate')
df_games['daysSinceLastGame'] = df_games.groupby('playerTeam')['gameDate'].diff().dt.days
df_games['daysSinceLastGame'] = df_games['daysSinceLastGame'].fillna(0)
games_unique = df_games[['gameId', 'daysSinceLastGame']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned = df_cleaned.drop(columns=['gameId'])

In [127]:
# won previous game
# Initialize the previous win column
df_games['won_previous'] = 0

# Iterate through each team and calculate if they won their previous game
for team in df_games['playerTeam'].unique():
    team_games = df_games[df_games['playerTeam'] == team].sort_values(by='gameDate')
    team_games['won_previous'] = (team_games['goalsFor'].shift(1) > team_games['goalsAgainst'].shift(1)).astype(int)
    df_games.loc[team_games.index, 'won_previous'] = team_games['won_previous']

games_unique = df_games[['gameId', 'won_previous']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned.drop(columns=['gameId'], inplace=True)

In [ ]:
# season phase
df_games['game_number'] = df_games.groupby('playerTeam').cumcount() + 1
df_games['total_games'] = df_games.groupby('playerTeam')['game_number'].transform('max')

# Determine the thresholds for early, mid, and late season
df_games['season_phase'] = 0
df_games.loc[df_games['game_number'] <= df_games['total_games'] / 3, 'season_phase'] = -1
df_games.loc[df_games['game_number'] > 2 * df_games['total_games'] / 3, 'season_phase'] = 1
games_unique = df_games[['gameId', 'season_phase']].drop_duplicates(subset='gameId')
df_cleaned = df_cleaned.merge(games_unique, left_on='game', right_on='gameId')
df_cleaned.drop(columns=['gameId'], inplace=True)

In [133]:

df_cleaned.to_csv('hockey_cleaned.csv', index=False)